# 렌탈횟수가 30회 이상인 유저의 리스트
* customer, rental table

In [1]:
import MySQLdb
import pandas as pd

db = MySQLdb.connect(
    'db.fastcamp.us',
    'root',
    'dkstncks',
    'sakila',
    charset='utf8',
)

In [3]:
pd.read_sql("SELECT * FROM customer;", db).tail(1)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
598,599,2,AUSTIN,CINTRON,AUSTIN.CINTRON@sakilacustomer.org,605,1,2006-02-14 22:04:37,2006-02-15 04:57:20


In [4]:
pd.read_sql("SELECT * FROM rental;", db).tail(1)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
16043,16049,2005-08-23 22:50:12,2666,393,2005-08-30 01:01:12,2,2006-02-15 21:30:53


### SQL(SUB QUERY)

In [20]:
# 일단 첫번째로
# 유저별로 몇 번 렌탈했을까? = > 지표를 뽑아내자
SQL_QUERY = """
    SELECT
        rental.customer_id,
        COUNT(rental.customer_id)
    FROM
        rental
    GROUP BY 1
"""

pd.read_sql(SQL_QUERY, db)

,customer_id,COUNT(rental.customer_id)
0,1,32
1,2,27
2,3,26
3,4,22
4,5,38
5,6,28
6,7,33
7,8,24
8,9,23
9,10,25


In [45]:
# 이후 새로운 쿼리문을 위 쿼리에 customer 테이블을 JOIN 시키자
# cutomer table과 매치시켜줘야함 => SQL을 다른 SQL에 조합가능
SQL_QUERY = """
    SELECT
        rental.customer_id,
        COUNT(rental.customer_id) 'total_count'
    FROM
        rental
    GROUP BY 1
    ;
"""

NEW_SQL_QUERY = """
    SELECT customer.*, rental_per_customer.total_count
    FROM 
        customer
        JOIN ({SQL_QUERY}) AS rental_per_customer
            ON customer.customer_id = rental_per_customer.customer_id
    WHERE rental_per_customer.total_count >= 30
    ;
""".format(
    SQL_QUERY=SQL_QUERY.replace(";", "")
)
# SQL_QUERY가 JOIN부분에 들어갈때 ;떄문에 끝났다고 인식하기때문에 오류가남
# 따라서 replace로 ;를 지워줘야함

# print(NEW_SQL_QUERY)

pd.read_sql(NEW_SQL_QUERY, db)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,total_count
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
1,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,38
2,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,33
3,15,1,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org,19,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
4,20,2,SHARON,ROBINSON,SHARON.ROBINSON@sakilacustomer.org,24,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
5,21,1,MICHELLE,CLARK,MICHELLE.CLARK@sakilacustomer.org,25,1,2006-02-14 22:04:36,2006-02-15 04:57:20,35
6,23,2,SARAH,LEWIS,SARAH.LEWIS@sakilacustomer.org,27,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
7,26,2,JESSICA,HALL,JESSICA.HALL@sakilacustomer.org,30,1,2006-02-14 22:04:36,2006-02-15 04:57:20,34
8,27,2,SHIRLEY,ALLEN,SHIRLEY.ALLEN@sakilacustomer.org,31,1,2006-02-14 22:04:36,2006-02-15 04:57:20,31
9,28,1,CYNTHIA,YOUNG,CYNTHIA.YOUNG@sakilacustomer.org,32,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32


### SQL(HAVING)
* HAVING 은 GROUP BY 연산이 끝난 후 조건이 적용
* WHERE 은 GROUP BY 연산 전에 조건이 적용되므로 오류가 발생하는것임

In [42]:
# 그룹바이를 하고나서야 카운트를 할수가 있는데 
# 그전에 WHERE를 적용하기 때문에 계산이 안됨
# WHERE -> HAVING 으로 해야함(연산이 끝난 상태에서 계산을 하는것임)
SQL_QUERY = """
    SELECT
        rental.customer_id,
        COUNT(rental.customer_id) 'total_count'
    FROM
        rental
    GROUP BY 1
    HAVING COUNT(rental.customer_id) >= 30
    ;
"""

NEW_SQL_QUERY = """
    SELECT customer.*, rental_per_customer.total_count
    FROM
        customer
        JOIN ({SQL_QUERY}) AS rental_per_customer
            ON customer.customer_id = rental_per_customer.customer_id
    ;
""".format(
    SQL_QUERY = SQL_QUERY.replace(";","")
)


pd.read_sql(NEW_SQL_QUERY, db)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,total_count
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
1,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,38
2,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,33
3,15,1,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org,19,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
4,20,2,SHARON,ROBINSON,SHARON.ROBINSON@sakilacustomer.org,24,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
5,21,1,MICHELLE,CLARK,MICHELLE.CLARK@sakilacustomer.org,25,1,2006-02-14 22:04:36,2006-02-15 04:57:20,35
6,23,2,SARAH,LEWIS,SARAH.LEWIS@sakilacustomer.org,27,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
7,26,2,JESSICA,HALL,JESSICA.HALL@sakilacustomer.org,30,1,2006-02-14 22:04:36,2006-02-15 04:57:20,34
8,27,2,SHIRLEY,ALLEN,SHIRLEY.ALLEN@sakilacustomer.org,31,1,2006-02-14 22:04:36,2006-02-15 04:57:20,31
9,28,1,CYNTHIA,YOUNG,CYNTHIA.YOUNG@sakilacustomer.org,32,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32


### pandas
* JOIN => merge
* WHERE => 행렬곱

In [47]:
customer_df = pd.read_sql("SELECT * FROM customer;", db)
customer_df.tail(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
597,598,1,WADE,DELVALLE,WADE.DELVALLE@sakilacustomer.org,604,1,2006-02-14 22:04:37,2006-02-15 04:57:20
598,599,2,AUSTIN,CINTRON,AUSTIN.CINTRON@sakilacustomer.org,605,1,2006-02-14 22:04:37,2006-02-15 04:57:20


In [48]:
rental_df = pd.read_sql("SELECT * FROM rental;", db)
rental_df.tail(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
16042,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-15 21:30:53
16043,16049,2005-08-23 22:50:12,2666,393,2005-08-30 01:01:12,2,2006-02-15 21:30:53


In [87]:
import numpy as np
rental_per_count_df = rental_df.groupby('customer_id').agg({
        'rental_id' : np.size,
    })
#rental_per_count_df.tail()

In [88]:
rental_per_count_df.rename(columns={'rental_id' : 'rental_count'}, inplace=True)
#rental_per_count_df.tail()

In [89]:
rental_per_count_df['customer_id'] = rental_per_count_df.index # merge시키기위해 index칼럼을 추가함
#rental_per_count_df.tail()

In [90]:
customer_with_rental_count_df = pd.merge(customer_df, rental_per_count_df, on='customer_id')
#customer_with_rental_count_df.tail()

In [82]:
customer_with_rental_count_df[customer_with_rental_count_df['rental_count'] >= 30]

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,rental_count
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,38
6,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,33
14,15,1,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org,19,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
19,20,2,SHARON,ROBINSON,SHARON.ROBINSON@sakilacustomer.org,24,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
20,21,1,MICHELLE,CLARK,MICHELLE.CLARK@sakilacustomer.org,25,1,2006-02-14 22:04:36,2006-02-15 04:57:20,35
22,23,2,SARAH,LEWIS,SARAH.LEWIS@sakilacustomer.org,27,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
25,26,2,JESSICA,HALL,JESSICA.HALL@sakilacustomer.org,30,1,2006-02-14 22:04:36,2006-02-15 04:57:20,34
26,27,2,SHIRLEY,ALLEN,SHIRLEY.ALLEN@sakilacustomer.org,31,1,2006-02-14 22:04:36,2006-02-15 04:57:20,31
27,28,1,CYNTHIA,YOUNG,CYNTHIA.YOUNG@sakilacustomer.org,32,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
